In [1]:
%load_ext autotime
#Web extractor
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup as BSoup

#Loop
import time

import pandas as pd
import numpy as np
import json
import datetime

# classi HTML degli elementi da prendere
classHotel = 'sr_item_content sr_item_content_slider_wrapper '
classNome = 'sr-hotel__name'
classCity = ' jq_tooltip district_link visited_link '
classLastbooking = 'rollover-s1 lastbooking'
classRoom = 'room_link'
classPrice = 'bui-price__weight-fix'
classLastPage = 'bui-pagination__item sr_pagination_item'
classRowRoom = 'roomrow entire_row_clickable'
classMaxperson = 'sr_max_occupancy sr_max_occ--inline'
xpathNextPage = '//*[@id="search_results_table"]/div[5]/div[1]/ul/li[3]/a'

In [8]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito') # usa modalità in incognito
options.add_experimental_option('prefs', {"profile.managed_default_content_settings.images": 2}) # non carica le immagini

driver = webdriver.Chrome(executable_path=r'C:/Users/loren/Documents/Scraping/chromedriver.exe', chrome_options=options)
url1 = 'https://www.booking.com/searchresults.en-gb.html?label=gen173nr-1DCAEoggJCAlhYSAlYBGhxiAEBmAEuwgEKd2luZG93cyAxMMgBD9gBA-gBAfgBApICAXmoAgM&sid=ec27a36aed44df134883ce3bf5a77299&'
url2 = '&class_interval=1&dest_id=1706&dest_type=region&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=region&room1=A%2CA&sb_price_type=total&search_selected=1&src=index&ss=England%2C United Kingdom&ss_all=0&ss_raw=eng&ssb=empty&sshis=0&nflt=ht_id%3D204%3B&rsf=ht_id-204&lsf=ht_id|204|5011&percent_htype_hotel=1&update_av=1'

time: 15.7 s


Creo dataset

In [3]:
hotelperpagina = 15

# creo un dataset con tante righe quanto gli hotel della città
HotelCity = pd.DataFrame(index = range(1800000), columns = ['Hotel', 'City', 'Room', 'Max Guests', 'Price','Room reinforcements',
                                                            'Check-in Date', 'Booking in last 6 hours', 'Scraping Date'])

time: 749 ms


In [6]:
k=0 # inizializzo indice dataset

time: 999 µs


In [9]:
# creo una lista con le date ogni 3 giorni da oggi (Scraping date) fino a tra un anno
for date in [(datetime.date.today() + datetime.timedelta(1)) + datetime.timedelta(n) for n in range(0, 365, 3)]:
    # costruisco url in base a checkin e checkout
    checkinmonth = 'checkin_month=' + str(date.month)
    checkinday = '&checkin_monthday=' + str(date.day)
    checkinyear = '&checkin_year=' + str(date.year)
    checkoutmonth = '&checkout_month=' + str((date + datetime.timedelta(3)).month)
    checkoutday = '&checkout_monthday=' + str((date + datetime.timedelta(3)).day)
    checkoutyear = '&checkout_year=' + str((date + datetime.timedelta(3)).year)
    HomePage = url1 + checkinmonth + checkinday + checkinyear + checkoutmonth + checkoutday + checkoutyear + url2
    driver.get(HomePage)

    htmlHome = BSoup(driver.page_source, 'html.parser') # scarico codice HTML
    pages = htmlHome.findAll('li', {'class':classLastPage}) # trovo l'elemento contenente l'ultima pagina per ciclo successivo
    lastPage = int(pages[-1].text[1:-1])
    for i in range(0, (lastPage-1)*hotelperpagina, hotelperpagina):
        # ciclo for basato su &offset (parte finale dell'url) che indica il numero dell'ultimo hotel della pagina precedente
        driver.get(HomePage+'&offset='+str(i))
        html = BSoup(driver.page_source, 'html.parser')
        Hotels = html.findAll('div', {'class':classHotel}) # trovo tutti i blocchi hotel nella pagina
        for el in Hotels: # ciclo for sulla lista di hotel
            HotelCity['Hotel'][k] = el.find('span', {'class':classNome}).text[1:-1] # trovo nome Hotel
            if ',' in el.find('a', {'class':classCity}).text[2:-14] # in alcuni casi è presente , se inserito anche quartiere
                HotelCity['City'][k] = el.find('a', {'class':classCity}).text[2:-14].split(', ')[1]
            else: # trovo nome Città
                HotelCity['City'][k] = el.find('a', {'class':classCity}).text[2:-14]
            try:
                HotelCity['Booking in last 6 hours'][k] = int(el.find('div', {'class':classLastbooking}).text.split()[1])
            except AttributeError: # trovo le prenotazione nelle ultime 6 ore (non sempre presente)
                pass
            HotelCity['Scraping Date'][k] = datetime.date.today()
            HotelCity['Check-in Date'][k] = date
            try:
                HotelCity['Room'][k] = el.find('span', {'class':classRoom}).text[1:-2].split('\n\n\n\n\n')[0] # trovo stanza
                HotelCity['Max Guests'][k] = int(el.find('div', {'class':classMaxperson}).text[1:-1].split()[-1]) # trovo n ospiti
                HotelCity['Price'][k] = float(el.find('tr', {'class':classRowRoom}).find('strong').text[4:-2].replace(',',''))
                # vado a prendere il prezzo dentro la riga della stanza e trovando l'unico oggetto strong
                try:
                    HotelCity['Room reinforcements'][k] = [a.text[1:-1] for a in el.findAll('sup')] # trovo optionals per la stanza
                except AttributeError:
                    pass
            except AttributeError:
                pass
            k=k+1
            
        if i in range(150, (lastPage-1)*hotelperpagina, hotelperpagina*10):
            driver.quit()
            driver = webdriver.Chrome(executable_path=r'C:/Users/loren/Documents/Scraping/chromedriver.exe', chrome_options=options)
            # chiudo e riapro il browser perchè consuma pian piano tutta la RAM
driver.quit()

time: 12h 32min 47s


# Controllo file

In [21]:
print(date, i, k-1, HotelCity.iloc[k-1])

2018-07-10 375 6034 Hotel                                               Heath Lodge Hotel nr NEC
City                                                              Bickenhill
Room                                                             Double Room
Max Guests                                                                 2
Price                                                                    210
Room reinforcements        [Breakfast included, FREE cancellation, No pre...
Check-in Date                                                     2018-07-10
Booking in last 6 hours                                                    7
Scraping Date                                                     2018-07-04
Name: 6034, dtype: object
time: 2.98 ms


## Controllo se ci sono righe con almeno un valore nullo

In [34]:
HotelCity[HotelCity.isnull().any(axis=1)]

,Hotel,City,Room,Price,Check-in Date,Booking in last 6 hours,Scraping Date
1,Holiday Inn Express Kettering Corby,Kettering,Family Room,373,2018-07-04,NaN,2018-07-04
2,The Clarence Boutique Hotel,Portsmouth,NaN,NaN,2018-07-04,NaN,2018-07-04
3,The Stafford London,London,Double Room,1387,2018-07-04,NaN,2018-07-04
5,Holiday Inn Express London - Epsom Downs,Epsom,Twin Room,481,2018-07-04,NaN,2018-07-04
6,Holiday Inn Express Birmingham Redditch,Redditch,Double Room,227,2018-07-04,NaN,2018-07-04
7,Holiday Inn Express Harlow,Harlow,Twin/Double Room,264,2018-07-04,NaN,2018-07-04
9,The Bear,Bath,NaN,NaN,2018-07-04,NaN,2018-07-04
13,Holiday Inn Express Peterborough,Peterborough,Double Room,377,2018-07-04,NaN,2018-07-04
18,Park Plaza Victoria London,London,Twin Room,993,2018-07-04,NaN,2018-07-04
19,The Bear,Bath,NaN,NaN,2018-07-04,NaN,2018-07-04


time: 1.88 s


Elimino righe con tutti valori nulli

In [ ]:
Hotel.drop_duplicates(how='all', keep=False, inplace=True)
HotelCity.reset_index(drop=True, inplace=True)

Aggiungo colonna ID

In [36]:
HotelCity['ID'] = HotelCity['Hotel'] + '_' + HotelCity['City']

time: 1.38 s


## Controllo hotel ripetuti

In [39]:
countH = HotelCity.groupby(['Check-in Date', 'ID'], as_index=False).size().to_frame('Count').sort_values(by='Count', ascending=False)
doubleH = countH[countH['Count'] > 1]
doubleH

Count
Check-in Date ID                                                       
2018-07-06    The Saracens Head Hotel_Great Dunmow                    3
2018-07-07    The Black Swan Inn_Norwich                              3
2018-07-06    Milton Hilltop Hotel_Carlisle                           3
2018-07-07    Best Western Victoria Palace_London                     3
              The Ancient Gatehouse_Wells                             3
2018-07-04    The Queens Hotel_Penzance                               3
2018-07-05    De Vere Jubilee Conference Centre_Nottingham            3
2018-07-09    Ely Hotel_Yateley                                       2
              Florence House Boutique Hotel and Restaurant_Po...      2
2018-07-06    Holiday Inn Express Harlow_Harlow                       2
              Holiday Inn Express Folkestone Channel Tunnel_F...      2
2018-07-04    The Chequers Hotel_Newbury                              2
              The Devon Hotel_Exeter                                  2
2018-07-06    Holiday Inn Express Braintree_Braintree                 2
2018-07-04    The Countryman's Inn_Hunton                             2
2018-07-09    Crowne Plaza Stratford-upon-Avon_Stratford-upon...      2
2018-07-04    The Heritage Hotel_Torquay                              2
              The Kings Arms and Royal Hotel – RelaxInnz_Goda...      2
              The Kings Arms_Fernhurst                                2
2018-07-07    Regent Hotel By The Lake_Ambleside                      2
2018-07-04    The Borough Lancaster_Lancaster                         2
              The Black Horse Inn_Canterbury                          2
              The Lawn Hotel_Reading                                  2
2018-07-06    Holiday Inn Express Manchester City Centre Aren...      2
              Holiday Inn Express Swindon City Centre_Swindon         2
2018-07-07    Milton Hilltop Hotel_Carlisle                           2
2018-07-04    Shiray Hotel_Blackpool                                  2
              Sorrento Hotel & Restaurant_Cambridge                   2
2018-07-07    Mountview Hotel Express_Saint Helier Jersey             2
2018-07-09    Highfield Hotel_Houghton le Spring                      2
...                                                                 ...
              The Chequers Hotel_Newbury                              2
2018-07-04    Bell_Thetford                                           2
2018-07-07    Bridge Hotel_Greenford                                  2
2018-07-09    Village Hotel London Watford_Elstree                    2
              The Borough Lancaster_Lancaster                         2
2018-07-06    Twinwoods Guestrooms_Milton Ernest                      2
2018-07-07    Florence House Boutique Hotel and Restaurant_Po...      2
2018-07-09    The Dovecote Inn_Laxton                                 2
2018-07-06    citizenM London Shoreditch_London                       2
              The Fox Inn Lower Oddington_Adlestrop                   2
              The Lawn Hotel_Reading                                  2
2018-07-09    The Western Hotel_St Ives                               2
2018-07-07    Bristol Marriott Hotel City Centre_Bristol              2
              Brookside Hotel & Restaurant_Chester                    2
2018-07-04    Doubletree By Hilton Hull_Hull                          2
2018-07-09    The White Hart Hotel_Welwyn                             2
              The Crown Hotel_Harrogate                               2
              The Kings Head Hotel_Great Bircham                      2
2018-07-04    Ely Hotel_Yateley                                       2
2018-07-06    The Arthouse Hotel_Liverpool                            2
              The Porterhouse grill & rooms_Oxford                    2
2018-07-07    Darlington Arms_Redhill                                 2
2018-07-04    Cranfield Management Development Centre_Cranfield       2
2018-07-09    The Manor at Weston 

time: 234 ms


Trovare indici degli hotel ripetuti

In [41]:
doppioni = Hotel[Hotel.duplicated(subset=['ID', 'Check-in Date'], keep=False)].sort_values('ID')
doppioni

C:\Users\loren\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  after removing the cwd from sys.path.


,Booking in last 6 hours,Check-in Date,City,Hotel,ID,Max Guests,Price,Room,Room reinforcements,Scraping Date
893,NaN,2018-07-04,Great Dunmow,The Saracens Head Hotel,The Saracens Head Hotel_Great Dunmow,NaN,NaN,NaN,NaN,2018-07-04
903,NaN,2018-07-04,Great Dunmow,The Saracens Head Hotel,The Saracens Head Hotel_Great Dunmow,NaN,NaN,NaN,NaN,2018-07-04
3120,NaN,2018-07-06,Great Dunmow,The Saracens Head Hotel,The Saracens Head Hotel_Great Dunmow,NaN,NaN,NaN,NaN,2018-07-04
3137,NaN,2018-07-06,Great Dunmow,The Saracens Head Hotel,The Saracens Head Hotel_Great Dunmow,NaN,NaN,NaN,NaN,2018-07-04
3147,NaN,2018-07-06,Great Dunmow,The Saracens Head Hotel,The Saracens Head Hotel_Great Dunmow,NaN,NaN,NaN,NaN,2018-07-04
4480,NaN,2018-07-07,Great Dunmow,The Saracens Head Hotel,The Saracens Head Hotel_Great Dunmow,NaN,NaN,NaN,NaN,2018-07-04
4854,NaN,2018-07-08,Great Dunmow,The Saracens Head Hotel,The Saracens Head Hotel_Great Dunmow,NaN,NaN,NaN,NaN,2018-07-04
4864,NaN,2018-07-08,Great Dunmow,The Saracens Head Hotel,The Saracens Head Hotel_Great Dunmow,NaN,NaN,NaN,NaN,2018-07-04
1879,NaN,2018-07-05,Norwich,The Black Swan Inn,The Black Swan Inn_Norwich,NaN,NaN,NaN,NaN,2018-07-04
1889,NaN,2018-07-05,Norwich,The Black Swan Inn,The Black Swan Inn_Norwich,NaN,NaN,NaN,NaN,2018-07-04


time: 15.6 s


# Salvataggio File

In [10]:
HotelCity.to_csv('Dataset Hotel\HotelPrice.csv', sep=';', index=False)

time: 5.69 s
